<a href="https://colab.research.google.com/github/nguyenkien1402/llamaindex-practices/blob/main/evaluation-pipeline-rag/rag_evaluation_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'MyDrive', 'llm-poc')
drive.mount(MOUNTPOINT)

Mounted at /content/gdrive


In [2]:
!pip install -r gdrive/MyDrive/llm-doc/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.3/806.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.4/422.4 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:0

In [1]:
from llama_index import download_loader
from llama_index import SimpleDirectoryReader

UnstructuredReader = download_loader('UnstructuredReader')

In [64]:
dir_reader = SimpleDirectoryReader('gdrive/MyDrive/llm-doc/data', file_extractor={
  ".pdf": UnstructuredReader(),
})
documents = dir_reader.load_data()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [59]:
import pandas as pd
from llama_index.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    LLMPredictor,
    Response,
    StorageContext,
    load_index_from_storage
)
from llama_index.node_parser import SimpleNodeParser
from llama_index.storage.index_store import SimpleIndexStore
from llama_index.prompts import Prompt

import chromadb
from llama_index.vector_stores import ChromaVectorStore

import openai
from llama_index.llms import OpenAI

openai.api_key = ""

# **Create Embedding Mode**


In [4]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="Represent this sentence for searching relevant passages: finance, investments, economics, real estate"
)

# **Initialize Service Context**

In [48]:
chroma_client.delete_collection('finance')

In [65]:
# create client and a new collection
chroma_client = chromadb.PersistentClient(path='gdrive/MyDrive/llm-doc/vector_store/')
chroma_collection = chroma_client.get_or_create_collection("finance_australia")

node_parser = SimpleNodeParser.from_defaults(chunk_size=1024, chunk_overlap=20)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo-16k", max_tokens=512, temperature=0.1),
                                               embed_model=model_norm,
                                               node_parser=node_parser)


**Approach 1: Naive Approach with VectorIndex Only**

In [66]:
indexid = 'finance_australia_index'
index_path = 'gdrive/MyDrive/llm-doc/index'
try:
  ## load index from storage
  print(f"Load {indexid} from local path")
  storage_context = StorageContext.from_defaults(vector_store=vector_store,
                                                  persist_dir=index_path)
  index = load_index_from_storage(storage_context=storage_context, index_id=indexid)
except Exception as e:
  print(str(e))
  print("Creating new index")
  storage_context = StorageContext.from_defaults(vector_store=vector_store)
  index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context,)
  index.set_index_id(indexid)
  index.storage_context.persist(persist_dir=index_path)

Load finance_australia_index from local path
Failed to load index with ID finance_australia_index
Creating new index


In [67]:
## testing index
query_engine = index.as_query_engine()
reponse = query_engine.query("How does the Nvidia's revenue in this report compare to the previous fiscal year, and what factors contributed to any changes?")
reponse.response

"There is no information provided in the given context about Nvidia's revenue or any changes in their revenue compared to the previous fiscal year. Therefore, it is not possible to answer the query based on the given context information."

**Step 1: Question Generation**

In [69]:
# use gpt4 for question generation
service_context_gpt4 = ServiceContext.from_defaults(llm=OpenAI(temperature=0.1, max_tokens=512, model="gpt-4"),
                                                    embed_model=model_norm,
                                                    node_parser=node_parser)
data_generator = DatasetGenerator.from_documents(
                      documents,
                      text_question_template=Prompt(
                      "A sample from the documents is below.\n"
                      "---------------------\n"
                      "{context_str}\n"
                      "---------------------\n"
                      "Using the documentation sample, carefully follow the instructions below:\n"
                      "{query_str}"
                      ),
                      question_gen_query=(
                          "You are a search pipeline evaluator. Using the papers provided, "
                          "you must create a list of summary questions and question/answer questions. "
                          "Limit the queries to the information supplied in the context.\n"
                          "Question: "
                      ),
                      service_context=service_context_gpt4)

In [70]:
eval_questions  = data_generator.generate_questions_from_nodes()
print(f"Generated {len(eval_questions )} questions.")

KeyboardInterrupt: ignored

In [71]:
eval_questions

NameError: ignored